<a href="https://colab.research.google.com/github/AdaAtess/CSCI317ProjectI/blob/main/CSCI317Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contribution Notes

This Colab notebook consists of resources, my thought process, bits of my code, example codes that could fix the mentioned issue, and my contribution notes.

Below is a small tutorial to see if I understand how to use TensorFlow in a basic way from a user's perspective.

In [ ]:
import tensorflow as tf
tf.add(1, 2).numpy()


3

In [ ]:
hello = tf.constant('Hello, TensorFlow!')
hello.numpy()


b'Hello, TensorFlow!'

The website of TensorFlow was a little complicated to find where to start from. There were a lot of sub-sections to go through as TensorFlow is very much well-established at this point and the community is unbelievably big.

I couldn't find any tutorials on how to conntribute or which issues/projects are the most crucial to contribute to. On the other hand, there were a lot of tutorials about how to use TensorFlow (beginner vs. experts) as well as free courses on Machine Learning. These seem quite interesting.

I kept exploring the website for contributions, there seemed to be a lot of ways to contribute -- groups, forums, blogs -- but it was pretty unclear where and how to start. When I found the "Contribute" sub-section under "Community," there was a "good first issue" and "contributions welcome" links. As I am an inexperienced open source software developer, I went along with "good first issue." However, when I followed the link, I saw that the issues were closed and have been closed for about 3 years. I saw some comments that suggested many people experienced the same problem as I did, as they were comments such as: "Hey, I'm a first time contributor, is this somewhere I can start from?" or "I really want to help but this issue seems closed, where can I contribute from" etc. 

I searched around a little bit more; however, I found it quite hard to start somewhere as the directions they point to were all sort of dead ends. 

Then, I decided to move to "Issues" section in GitHub repo of TensorFlow and decided to tackle a problem there.

Overall, I would say it was a frustrating experience. You want to contribute and as you go on the website and follow the directions, you end up at dead ends. It sort of takes away from your eagerness and I have seen people voicing the same issue as I have experienced.

In TensorFlow GitHub repo, I found this issue about an additional math operations functionality. TF1 is wanted to be supported by hex to see tensors in hex version. TF2 supports this feature but TF1 doesn't. The person who started this issue was using TF1 did not necessarily want to migrate to TF2 and wanted an easier solution. Here is the github code pointed in discussion "[Feature] To support casting float32 to int32 in HEX format #30493":
https://github.com/tensorflow/tensorflow/issues/30493

This is where the code seems to create a problem:

https://github.com/tensorflow/tensorflow/blob/v2.7.0/tensorflow/python/ops/math_ops.py#L938-L1007


In [ ]:
@tf_export("cast", "dtypes.cast")
@dispatch.register_unary_elementwise_api
@dispatch.add_dispatch_support
def cast(x, dtype, name=None):
  """Casts a tensor to a new type.
  The operation casts `x` (in case of `Tensor`) or `x.values`
  (in case of `SparseTensor` or `IndexedSlices`) to `dtype`.
  For example:
  >>> x = tf.constant([1.8, 2.2], dtype=tf.float32)
  >>> tf.cast(x, tf.int32)
  <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>
  Notice `tf.cast` has an alias `tf.dtypes.cast`:
  >>> x = tf.constant([1.8, 2.2], dtype=tf.float32)
  >>> tf.dtypes.cast(x, tf.int32)
  <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>
  The operation supports data types (for `x` and `dtype`) of
  `uint8`, `uint16`, `uint32`, `uint64`, `int8`, `int16`, `int32`, `int64`,
  `float16`, `float32`, `float64`, `complex64`, `complex128`, `bfloat16`.
  In case of casting from complex types (`complex64`, `complex128`) to real
  types, only the real part of `x` is returned. In case of casting from real
  types to complex types (`complex64`, `complex128`), the imaginary part of the
  returned value is set to `0`. The handling of complex types here matches the
  behavior of numpy.
  Note casting nan and inf values to integral types has undefined behavior.
  Args:
    x: A `Tensor` or `SparseTensor` or `IndexedSlices` of numeric type. It could
      be `uint8`, `uint16`, `uint32`, `uint64`, `int8`, `int16`, `int32`,
      `int64`, `float16`, `float32`, `float64`, `complex64`, `complex128`,
      `bfloat16`.
    dtype: The destination type. The list of supported dtypes is the same as
      `x`.
    name: A name for the operation (optional).
  Returns:
    A `Tensor` or `SparseTensor` or `IndexedSlices` with same shape as `x` and
      same type as `dtype`.
  Raises:
    TypeError: If `x` cannot be cast to the `dtype`.
  """
  base_type = dtypes.as_dtype(dtype).base_dtype
  if isinstance(x,
                (ops.Tensor, _resource_variable_type)) and base_type == x.dtype:
    return x
  with ops.name_scope(name, "Cast", [x]) as name:
    if isinstance(x, sparse_tensor.SparseTensor):
      values_cast = cast(x.values, base_type, name=name)
      x = sparse_tensor.SparseTensor(x.indices, values_cast, x.dense_shape)
    elif isinstance(x, ops.IndexedSlices):
      values_cast = cast(x.values, base_type, name=name)
      x = ops.IndexedSlices(values_cast, x.indices, x.dense_shape)
    else:
      # TODO(josh11b): If x is not already a Tensor, we could return
      # ops.convert_to_tensor(x, dtype=dtype, ...)  here, but that
      # allows some conversions that cast() can't do, e.g. casting numbers to
      # strings.
      x = ops.convert_to_tensor(x, name="x")
      if x.dtype.base_dtype != base_type:
        x = gen_math_ops.cast(x, base_type, name=name)
    if x.dtype.is_complex and base_type.is_floating:
      logging.warn("Casting complex to real discards imaginary part.")
    return x

NameError: ignored

Example solution on TensorFlow website published more recently compared to the issue's opening.
https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/bitcast

In [ ]:
tf.bitcast(
    input, type, name=None
)

NameError: ignored

Given a tensor input, this operation returns a tensor that has the same buffer data as input with datatype type.

If the input datatype T is larger than the output datatype type then the shape changes from [...] to [..., sizeof(T)/sizeof(type)].

If T is smaller than type, the operator requires that the rightmost dimension be equal to sizeof(type)/sizeof(T). The shape then goes from [..., sizeof(type)/sizeof(T)] to [...].

tf.bitcast() and tf.cast() work differently when real dtype is casted as a complex dtype (e.g. tf.complex64 or tf.complex128) as tf.cast() make imaginary part 0 while tf.bitcast() gives module error. For example,

**Example 1:**

In [ ]:
a = [1., 2., 3.]
equality_bitcast = tf.bitcast(a,tf.complex128)

equality_cast = tf.cast(a,tf.complex128)
print(equality_cast)

**Example 2:**

In [ ]:
>>> tf.bitcast(tf.constant(0xffffffff, dtype=tf.uint32), tf.uint8)
<tf.Tensor: ... shape=(4,), dtype=uint8, numpy=array([255, 255, 255, 255], dtype=uint8)>


**Example 3:**

In [ ]:
>>> x = [1., 2., 3.]
>>> y = [0., 2., 3.]
>>> equality= tf.equal(x,y)
>>> equality_cast = tf.cast(equality,tf.float32)
>>> equality_bitcast = tf.bitcast(equality_cast,tf.uint8)
>>> print(equality)
tf.Tensor([False True True], shape=(3,), dtype=bool)
>>> print(equality_cast)
tf.Tensor([0. 1. 1.], shape=(3,), dtype=float32)
>>> print(equality_bitcast)
tf.Tensor(
[[ 0 0 0 0]
 [ 0 0 128 63]
 [ 0 0 128 63]], shape=(3, 4), dtype=uint8)

However, the person's problem was about TF1 and TF2 creating differences and hence hex not being able to be used as wanted. I was doing some research and thinking on how to solve this issue, I came across a very recent article that was published with the most recent update of TensorFlow which could solve this problem.
https://www.tensorflow.org/api_docs/python/tf/io/decode_raw

The possible solution suggested in the article:

In [ ]:
tf.io.decode_raw(
    input_bytes, out_type, little_endian=True, fixed_length=None, name=None
)

NameError: ignored

In [ ]:
tf.io.decode_raw(tf.constant("1"), tf.uint8)

tf.io.decode_raw(tf.constant("1,2"), tf.uint8)


<tf.Tensor: shape=(3,), dtype=uint8, numpy=array([49, 44, 50], dtype=uint8)>

In [ ]:
tf.io.decode_raw(tf.constant(["1","2"]), tf.uint8).shape

tf.io.decode_raw(tf.constant([["1"],["2"]]), tf.uint8).shape


TensorShape([2, 1, 1])

In [ ]:
tf.io.decode_raw(tf.constant("123"), tf.uint8)

tf.io.decode_raw(tf.constant("1234"), tf.uint8)

# chuncked output
tf.io.decode_raw(tf.constant("12"), tf.uint16)

tf.io.decode_raw(tf.constant("1234"), tf.uint16)

# int64 output
tf.io.decode_raw(tf.constant("12345678"), tf.int64)

tf.io.decode_raw(tf.constant("1234567887654321"), tf.int64)


<tf.Tensor: shape=(2,), dtype=int64, numpy=array([4050765991979987505, 3544952156018063160])>

In [ ]:
tf.io.decode_raw(tf.constant("\x0a\x0b"), tf.int16)

hex(2826)

tf.io.decode_raw(tf.constant("\x0a\x0b"), tf.int16, little_endian=False)

hex(2571)


'0xa0b'

In [ ]:
tf.io.decode_raw(tf.constant([["1"],["23"]]), tf.uint8, fixed_length=4)




<tf.Tensor: shape=(2, 1, 4), dtype=uint8, numpy=
array([[[49,  0,  0,  0]],

       [[50, 51,  0,  0]]], dtype=uint8)>

In [ ]:
tf.io.decode_raw(tf.constant(["1212"]), tf.uint16, fixed_length=4)


<tf.Tensor: shape=(1, 2), dtype=uint16, numpy=array([[12849, 12849]], dtype=uint16)>

In [ ]:
x=''.join([chr(1), chr(2), chr(3), chr(4)])
tf.io.decode_raw(x, tf.uint16, fixed_length=2)

hex(513)


'0x201'

In [ ]:
x=''.join([chr(1), chr(2), chr(3), chr(4)])
tf.io.decode_raw(x, tf.uint16, fixed_length=2, little_endian=False)

hex(258)


'0x102'

In [ ]:
x = ["12345678", "87654321"]
tf.io.decode_raw(x, tf.int16)



tf.io.decode_raw(x, tf.int16, fixed_length=len(x[0]))




<tf.Tensor: shape=(2, 4), dtype=int16, numpy=
array([[12849, 13363, 13877, 14391],
       [14136, 13622, 13108, 12594]], dtype=int16)>

Code  below is in the source code of TF2. I realized the person was asking for help without migrating to TF2. I think this could be of help. I saw that there is a recent functionality called -- tf2.convert_to_tf1 -- that could be used after using the below to achieve hex code. Or the above functions that would work in TF1 as well. TensorFlow seem to have fixed this issue. I was not sure if I should reply to the person and point to these resources as it has been a while since this issue was discussed as it was opened in 2019 and nothing has been seem to be published for some time.

Note. These source codes were pretty hard to find. The documentation inside the files are very detailed and easy to follow; however, the files themselves are hard to find and they don't necessarily have explanations on why they exist and what their specific purpose in the general source code is. The general documentation isn't as clear as much as it should be in my opinion.

Code below is from: https://github.com/tensorflow/tensorflow/blob/v2.8.0/tensorflow/python/ops/parsing_ops.py#L842-L974

In [ ]:
@tf_export("io.decode_raw", v1=[])
@dispatch.add_dispatch_support
def decode_raw(input_bytes,
               out_type,
               little_endian=True,
               fixed_length=None,
               name=None):
  r"""Convert raw bytes from input tensor into numeric tensors.
  Every component of the input tensor is interpreted as a sequence of bytes.
  These bytes are then decoded as numbers in the format specified by `out_type`.
  >>> tf.io.decode_raw(tf.constant("1"), tf.uint8)
  <tf.Tensor: shape=(1,), dtype=uint8, numpy=array([49], dtype=uint8)>
  >>> tf.io.decode_raw(tf.constant("1,2"), tf.uint8)
  <tf.Tensor: shape=(3,), dtype=uint8, numpy=array([49, 44, 50], dtype=uint8)>
  Note that the rank of the output tensor is always one more than the input one:
  >>> tf.io.decode_raw(tf.constant(["1","2"]), tf.uint8).shape
  TensorShape([2, 1])
  >>> tf.io.decode_raw(tf.constant([["1"],["2"]]), tf.uint8).shape
  TensorShape([2, 1, 1])
  This is because each byte in the input is converted to a new value on the
  output (if output type is `uint8` or `int8`, otherwise chunks of inputs get
  coverted to a new value):
  >>> tf.io.decode_raw(tf.constant("123"), tf.uint8)
  <tf.Tensor: shape=(3,), dtype=uint8, numpy=array([49, 50, 51], dtype=uint8)>
  >>> tf.io.decode_raw(tf.constant("1234"), tf.uint8)
  <tf.Tensor: shape=(4,), dtype=uint8, numpy=array([49, 50, 51, 52], ...
  >>> # chuncked output
  >>> tf.io.decode_raw(tf.constant("12"), tf.uint16)
  <tf.Tensor: shape=(1,), dtype=uint16, numpy=array([12849], dtype=uint16)>
  >>> tf.io.decode_raw(tf.constant("1234"), tf.uint16)
  <tf.Tensor: shape=(2,), dtype=uint16, numpy=array([12849, 13363], ...
  >>> # int64 output
  >>> tf.io.decode_raw(tf.constant("12345678"), tf.int64)
  <tf.Tensor: ... numpy=array([4050765991979987505])>
  >>> tf.io.decode_raw(tf.constant("1234567887654321"), tf.int64)
  <tf.Tensor: ... numpy=array([4050765991979987505, 3544952156018063160])>
  The operation allows specifying endianness via the `little_endian` parameter.
  >>> tf.io.decode_raw(tf.constant("\x0a\x0b"), tf.int16)
  <tf.Tensor: shape=(1,), dtype=int16, numpy=array([2826], dtype=int16)>
  >>> hex(2826)
  '0xb0a'
  >>> tf.io.decode_raw(tf.constant("\x0a\x0b"), tf.int16, little_endian=False)
  <tf.Tensor: shape=(1,), dtype=int16, numpy=array([2571], dtype=int16)>
  >>> hex(2571)
  '0xa0b'
  If the elements of `input_bytes` are of different length, you must specify
  `fixed_length`:
  >>> tf.io.decode_raw(tf.constant([["1"],["23"]]), tf.uint8, fixed_length=4)
  <tf.Tensor: shape=(2, 1, 4), dtype=uint8, numpy=
  array([[[49,  0,  0,  0]],
         [[50, 51,  0,  0]]], dtype=uint8)>
  If the `fixed_length` value is larger that the length of the `out_type` dtype,
  multiple values are generated:
  >>> tf.io.decode_raw(tf.constant(["1212"]), tf.uint16, fixed_length=4)
  <tf.Tensor: shape=(1, 2), dtype=uint16, numpy=array([[12849, 12849]], ...
  If the input value is larger than `fixed_length`, it is truncated:
  >>> x=''.join([chr(1), chr(2), chr(3), chr(4)])
  >>> tf.io.decode_raw(x, tf.uint16, fixed_length=2)
  <tf.Tensor: shape=(1,), dtype=uint16, numpy=array([513], dtype=uint16)>
  >>> hex(513)
  '0x201'
  If `little_endian` and `fixed_length` are specified, truncation to the fixed
  length occurs before endianness conversion:
  >>> x=''.join([chr(1), chr(2), chr(3), chr(4)])
  >>> tf.io.decode_raw(x, tf.uint16, fixed_length=2, little_endian=False)
  <tf.Tensor: shape=(1,), dtype=uint16, numpy=array([258], dtype=uint16)>
  >>> hex(258)
  '0x102'
  If input values all have the same length, then specifying `fixed_length`
  equal to the size of the strings should not change output:
  >>> x = ["12345678", "87654321"]
  >>> tf.io.decode_raw(x, tf.int16)
  <tf.Tensor: shape=(2, 4), dtype=int16, numpy=
  array([[12849, 13363, 13877, 14391],
         [14136, 13622, 13108, 12594]], dtype=int16)>
  >>> tf.io.decode_raw(x, tf.int16, fixed_length=len(x[0]))
  <tf.Tensor: shape=(2, 4), dtype=int16, numpy=
  array([[12849, 13363, 13877, 14391],
         [14136, 13622, 13108, 12594]], dtype=int16)>
  Args:
    input_bytes:
      Each element of the input Tensor is converted to an array of bytes.
      Currently, this must be a tensor of strings (bytes), although semantically
      the operation should support any input.
    out_type:
      `DType` of the output. Acceptable types are `half`, `float`, `double`,
      `int32`, `uint16`, `uint8`, `int16`, `int8`, `int64`.
    little_endian:
      Whether the `input_bytes` data is in little-endian format. Data will be
      converted into host byte order if necessary.
    fixed_length:
      If set, the first `fixed_length` bytes of each element will be converted.
      Data will be zero-padded or truncated to the specified length.
      `fixed_length` must be a multiple of the size of `out_type`.
      `fixed_length` must be specified if the elements of `input_bytes` are of
      variable length.
    name: A name for the operation (optional).
  Returns:
    A `Tensor` object storing the decoded bytes.
  """
  if fixed_length is not None:
    return gen_parsing_ops.decode_padded_raw(
        input_bytes,
        fixed_length=fixed_length,
        out_type=out_type,
        little_endian=little_endian,
        name=name)
  else:
    return gen_parsing_ops.decode_raw(
        input_bytes, out_type, little_endian=little_endian, name=name)

Overall, finding a place to start from was a hard and frustrating process. It is not as welcoming as they make it out to be for first-time contributors. However, there is a plethora of resources and places of discussion, the community seems to be tight and helpful. It was nice to see people interact, not dismiss questions/comments as "unworthy" or "basic", and be of help to everyone.